# Model Analysis
____

In this notebook, I will be analysis the the two models that were trained in the Modeling.ipynb notebook and how they score on the 24 diffrent classes that can be shown in an image

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
import cv2


from glob import glob
import os

import sys
#sys.path.append(os.path.abspath('u_net_model.py'))
#import module

2023-06-11 15:33:56.164766: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
os.path.abspath('drone_image_segmentation')

'/Users/andresaguilar/Desktop/dsi/projects/drone-image-segmentation/notebooks/drone_image_segmentation'

In [2]:
os.path.abspath('u_net_model.py')

'/Users/andresaguilar/Desktop/dsi/projects/project_6/notebooks/u_net_model.py'

### Loading Data

I will load in the preprocessed data set that had resized the images and masks

In [2]:
img_folder = '../semantic_drone_dataset/processed/images/'
mask_folder = '../semantic_drone_dataset/processed/label_images/'

img_files = np.sort(os.listdir(img_folder))
mask_files = np.sort(os.listdir(mask_folder))

images = create_image_array(img_folder, img_files)
masks = create_image_array(mask_folder, mask_files)

FileNotFoundError: [Errno 2] No such file or directory: '../semantic_drone_dataset/processed/images/'

In [4]:
#
true_masks = list(map(mask_classes, masks))

In [5]:
model = unet_model(24)
model.load_weights('seg_model_scc.h5')

In [6]:
preds = []
preds_rgb = []
for img in images:
    pred = model.predict(img[tf.newaxis, ...], verbose = 0)
    pred = mask_decoding(pred)[0]
    pred_rgb = mask_rgb(pred)
    preds.append(pred)
    preds_rgb.append(pred_rgb)

In [19]:
model.predict(images[0][tf.newaxis, ...], verbose = 0)[0][0][0]

array([0.00934788, 0.00566339, 0.00077261, 0.00036697, 0.04810522,
       0.00150536, 0.01209637, 0.027877  , 0.00188291, 0.04949784,
       0.04890022, 0.02290533, 0.0395032 , 0.01320803, 0.02588791,
       0.03442249, 0.00470291, 0.1831709 , 0.01378493, 0.0025609 ,
       0.00244085, 0.2321787 , 0.21217003, 0.00704809], dtype=float32)

In [24]:
series = []
for i in range(len(images)):
    series.append([images[i], true_masks[i], preds[i]])
    
series_rgb = []
for i in range(len(images)):
    series_rgb.append([images[i], masks[i], preds_rgb[i]])

In [28]:
class_names = list(pd.read_csv('./semantic_drone_dataset/class_dict.csv')['name'])

In [34]:
class_iou = {}
for i in range(24):
    column = class_names[i] + '_iou_score'
    class_iou[column] =  []
    iou = tf.keras.metrics.IoU(num_classes=24, target_class_ids=[i])
    for j in range(len(series)):
        iou.update_state(series[j][1], series[j][2])
        class_iou[column].append( iou.result().numpy() )
        iou.reset_state()

In [35]:
df = pd.DataFrame(class_iou)
df.head()

,unlabelediou_score,paved-areaiou_score,dirtiou_score,grassiou_score,graveliou_score,wateriou_score,rocksiou_score,pooliou_score,vegetationiou_score,roofiou_score,...,fence-poleiou_score,personiou_score,dogiou_score,cariou_score,bicycleiou_score,treeiou_score,bald-treeiou_score,ar-markeriou_score,obstacleiou_score,conflictingiou_score
0,0.0,0.0,0.0,0.0,0.001337,0.0,0.0,0.0,0.009917,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.004878,0.0,0.00000,0.016253,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000131,0.000492,...,0.0,0.0,0.0,0.000000,0.0,0.001290,0.0,0.00572,0.035295,0.0
2,0.0,0.0,0.0,0.0,0.000651,0.0,0.0,0.0,0.003152,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.008375,0.0,0.00000,0.022531,0.0
3,0.0,0.0,0.0,0.0,0.000583,0.0,0.0,0.0,0.001728,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.012039,0.0
4,0.0,0.0,0.0,0.0,0.000263,0.0,0.0,0.0,0.004961,0.060553,...,0.0,0.0,0.0,0.034363,0.0,0.002639,0.0,0.00000,0.027395,0.0


In [46]:
df.mean()

unlabelediou_score      6.329114e-06
paved-areaiou_score     7.778825e-06
dirtiou_score           0.000000e+00
grassiou_score          2.562473e-07
graveliou_score         4.688045e-03
wateriou_score          6.447814e-05
rocksiou_score          0.000000e+00
pooliou_score           2.470486e-05
vegetationiou_score     1.628821e-02
roofiou_score           3.737081e-02
walliou_score           6.976093e-03
windowiou_score         6.484529e-03
dooriou_score           0.000000e+00
fenceiou_score          0.000000e+00
fence-poleiou_score     2.252252e-05
personiou_score         5.480384e-04
dogiou_score            0.000000e+00
cariou_score            1.505645e-02
bicycleiou_score        0.000000e+00
treeiou_score           2.238713e-03
bald-treeiou_score      0.000000e+00
ar-markeriou_score      4.034429e-03
obstacleiou_score       3.115117e-02
conflictingiou_score    0.000000e+00
dtype: float32

In [43]:
ls = df.describe().T
ls[ls['max'] > 0]['mean'].mean()

0.007810160099632313

In [44]:
list(pd.read_csv('./semantic_drone_dataset/class_dict.csv')['name'])

['unlabeled',
 'paved-area',
 'dirt',
 'grass',
 'gravel',
 'water',
 'rocks',
 'pool',
 'vegetation',
 'roof',
 'wall',
 'window',
 'door',
 'fence',
 'fence-pole',
 'person',
 'dog',
 'car',
 'bicycle',
 'tree',
 'bald-tree',
 'ar-marker',
 'obstacle',
 'conflicting']